#### Lecture 0. OT

- 준 근현대사 강의 : 굵직한 CV 분야의 흐름을 파악할 수 있도록
- 딥러닝 기반 기술만
- 영어 강의 자료 : 영어 표현에 익숙해지기, 용어 통일, 스스로 논문을 분석하는 기본기

#### Lecture 1. Image classification

1. AI in CV
- AI : 사람의 지능을 computer system으로 구현 (visual perception, speech recognition, decision-making, translation between lanuages)
- 인간 지능 : 오감을 활용하여 지각 능력을 획득하고 이를 통해 사고가 발달함
- perception => (input, output) data
- 이러한 지각능력에서 가장 큰 비중을 차지하는 것이 시각
- data 를 ai algorithm을 통해 representation 으로 변환
- computer graphics (rendering) // computer vision (inverse rendering)
- 우리의 지각 능력도 불완전하며 visual perception 에서 이것에 대한 이해가 선행되어야 불완전성을 극복할 수 있음 (ex thatcher illusion)
- implementation
    - machine learning : input > feature extraction > classification > output
    - deep learning : input > feature extraction + classification > output
        - 사람이 특징을 추출하는데 들어가는 bias, dl의 gd가 더 특징 추출을 잘함
- cv 분야 수요가 높음 ( google scholar CV 논문이 5위권 )

2. Image classification
- classifier : image 와 category level의 mapping
- k-NN(Nearest Neigbors)
    - 모든 데이터를 보유하고 있다면 모든 분류 문제를 풀수 있음
    - 비슷한 데이터의 라벨로부터 추측
    - 시간과 공간 복잡도의 한계, vision data 간의 유사도 정의가 어려움
    - neural network 는 이 복잡도를 제한된 복잡도의 딥러닝 모델에 압축하는 것

3. CNN
- single fully connected layer : 가중치 matrix를 영상에 맞게 reshape 을 하면 입력 영상과 동일한 사이즈의 template으로 생각할 수 있으며 template과의 내적으로 score 추출
    - 구조가 단순해서 평균 이미지 정도만 표현이 가능
    - test set이 training과 다르면 잘못된 결론을 도출할 가능성이 높음
- convolution neural network : locally connected neural network
    - 지역적 특징을 추출한 뒤 sliding window처럼 전 영역을 순회하면서 connection 공유
    - 효과적인 feature 추출, parameter를 공유하여 과적합이 방지됨
    - 많은 CV task의 backbone network로 기능함

4. CNN Architectures
- 첫 CNN (Yann LeCun 1998) : Conv-Pool-Conv-Pool-FC-FC
- AlexNet(2012) : 7 layers, 60 m parameters, ReLU, dropout 사용, GPU 2개 병령 사용
    - conv-(LRN-local response normalization)-ReLU-max_pool 중첩
    - max_pool 된 데이터를 (avgPool 또는)flatten으로 벡터화 후 FC
    - LRN은 사양 추세, 11x11 filter size 최근에는 더 작은 filter size 사용
    - receptive field : layer의 하나의 값을 결정하기 위해 필요한 input data 영역
- VGGNet(2014) : 16/19 layers, 3x3 conv filter, 2x2 max pool
    - 깊고 단순한 architecture, 일반화 성능 우수

#### Lecture 2. Annotation data efficient learning

1. Data augmentation
- 사람이 camera로 찍은 data는 biased 될 수 밖에 없음
- training dataset 이 real data distribution 을 표현할 수 없음
- augmented data 는 이 간극을 메워보려는 하나의 시도
- rotation, flip, crop, brightness, affine transformation(선형 변환)와 같은 image transformation 이 대표적으로 openCV, NumPy에 구현됨
- cutmix : 데이터를 잘라서 붙이고 label도 가중치로 합성
- RandAugment : 랜덤하게 augmentation 기법들을 sampling 함
    - policy : random sampling을 통해 얻은 N개의 augmentation 기법

2. leveraging pre-trained information
- transfer learning : 한 데이터셋에서 배운 지식을 다른 데이터셋에 활용
    - 다른 데이터셋을 통해 얻은 모델에서 FC 를 제거하고 Convolution 부분만 다른 FC를 합치고 re-train 함
    - FC만 가중치를 변경하는 방법1, Convolution은 낮은lr, FC는 높은 lr적용하는 방법 2
- knowledge distillation : 이미 학습된 teacher model의 지식을 student model에 전달
    - 원래는 model compression에 쓰이던 기법
    - unlabeled dataset에 pseudo-label을 generating 하는 기법으로 사용됨
    - 동일한 input을 teacher model(pre-trained)과 student model에 통과시킨 output끼리 KL div.loss 를 측정하여 backpropagation을 통해 student model 학습
    - loss 계산 시에는 soft label이용(확률의 실수값) // cf) hard label : one-hot vec
    - softmax 계산 시에는 각각의 입력값을 temperature T로 나누어 출력값을 smooth하게
    - teacher model의 soft label의 각각의 값은 KL-divergence 의 차이를 계산하기 위한 값으로만 기능할 뿐(label이 없으므로) semantic information 은 무시함
    - distillation loss : KLdiv로 계산, teacher/student network 사이의 차이
    - student loss : cross-entropy로 계산, ground truth 학습
    - 둘의 가중합으로 backpropagation이 이루어짐

3. leveraging unlabeled dataset for training
- semi-supervised learning : label된 dataset으로 model을 학습한 후 unlabel된 dataset을 입력으로 하여 label을 예측한 후(pseudo-label) 전체 dataset으로 학습
- self-training : augmentation + teacher/student learning + semi-supervised learning
    - imageNet의 1M labeled data로 teacher model 학습
    - 300M unlabeled data를 pseudo-labeled data로 변환
    - 전체 dataset으로 RandAugment 까지 수행한 방대한 데이터로 student model 학습
    - student model을 teacher model로 삼아  다시 unlabeled data > pseudo dataset
    - 다시 전체 dataset으로 new student model을 학습하는 과정을 반복
    - 이 때 student model이 점진적으로 커지는 방식으로 진행
    

#### peer session

데이콘 인공지능 비트 트레이더 경진대회 출전하기로 함
https://dacon.io/competitions/official/235709/overview/rules#rule-info